This notebook requires:
* the North Sea test case from the [getm-setups repository](https://github.com/getm-model/setups)
* the [TPXO9 atlas](https://www.tpxo.net/global/tpxo9-atlas) to calculate tides at the open boundaries

In [ ]:
getm_setups_dir = '../../../getm-setups'
igotm_data_dirs = ('/server/data', '../../../igotm/data')

In [ ]:
import datetime
import os.path

import pygetm
import pygetm.domain
import pygetm.input
import pygetm.legacy
import pygetm.input.tpxo
import pygetm.airsea
import pygetm.mixing

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

In [ ]:
# Load bathymetry and create simulation
domain = pygetm.legacy.domain_from_topo(os.path.join(getm_setups_dir, 'NorthSea/Topo/NS6nm.v01.nc'), nlev=20, z0_const=0.001)
pygetm.legacy.load_bdyinfo(domain, os.path.join(getm_setups_dir, 'NorthSea/bdyinfo.dat'))
sim = pygetm.Simulation(domain, runtype=2, apply_bottom_friction=True, advection_scheme=1)
print('Maximum dt = %.3f s' % domain.maxdt)

# ensure ho and hn are up to date and identical
domain.do_vertical()
matplotlib.pyplot.subplots()
domain.T.ho.isel(z=-1).plot()

In [ ]:
# Load forcing
igotm_data_dir = next(filter(os.path.isdir, igotm_data_dirs))

# Meteorology from ERA
era_path = os.path.join(igotm_data_dir, 'ERA-interim/2016.nc')
tcc = domain.T.map(pygetm.input.get_from_nc(era_path, 'tcc', preprocess=lambda ds: ds.isel(time=slice(4, -4))))
t2m = domain.T.map(pygetm.input.get_from_nc(era_path, 't2m', preprocess=lambda ds: ds.isel(time=slice(4, -4))))
d2m = domain.T.map(pygetm.input.get_from_nc(era_path, 'd2m', preprocess=lambda ds: ds.isel(time=slice(4, -4))))
sp = domain.T.map(pygetm.input.get_from_nc(era_path, 'sp', preprocess=lambda ds: ds.isel(time=slice(4, -4))))
u10 = domain.T.map(pygetm.input.get_from_nc(era_path, 'u10', preprocess=lambda ds: ds.isel(time=slice(4, -4))))
v10 = domain.T.map(pygetm.input.get_from_nc(era_path, 'v10', preprocess=lambda ds: ds.isel(time=slice(4, -4))))

In [ ]:
# Create temperature and salinity tracers
t = domain.T.array(fill=10, is_3d=True)
s = domain.T.array(fill=35, is_3d=True)
sst = t.isel(z=-1)
t_source = domain.T.array(fill=0., is_3d=True)
shf = t_source.isel(z=-1)

# Set up initial plot of SST (to be animated later)
ref_date = datetime.datetime(2016, 1, 1)
fig, (ax1, ax2) = matplotlib.pyplot.subplots(ncols=2, figsize=(12, 5))
pc = ax1.pcolormesh(domain.T.loni, domain.T.lati, sst.ma, vmin=5, vmax=20)
cb = fig.colorbar(pc, ax=ax1)
cb.set_label('sea surface temperature (m)')
pc_hf = ax2.pcolormesh(domain.T.loni, domain.T.lati, shf.ma, vmin=-100, vmax=100)
cb = fig.colorbar(pc_hf, ax=ax2)
cb.set_label('surface heat flux (W m-2)')
title = fig.suptitle(ref_date.strftime('%Y-%m-%d %H:%M'))

In [ ]:
inputs = t2m, d2m, sp, u10, v10

vdif = pygetm.VerticalDiffusion(t.grid, cnpar=1.)
turb = pygetm.mixing.Turbulence(domain)

cp = 3985.
rho0 = 1025.

timestep = 18000.
hf_scale_factor = timestep / (rho0 * cp)
timedelta = datetime.timedelta(seconds=timestep)
date = ref_date
for itime in range(60 *24 * 30):
    for variable in inputs:
        variable.update(date)

    es, ea, qs, qa, rhoa = pygetm.airsea.humidity(domain.T, 3, d2m - 273.15, sp, sst, t2m - 273.15)
    tausx, tausy, qe, qh = pygetm.airsea.airsea_fluxes(domain.T, 1, sst + 273.15, t2m, u10, v10, rhoa, qs, qa)
    shf.all_values[...] = (qe.all_values + qh.all_values)
    s = t_source * hf_scale_factor
    vdif(turb.nuh, timestep, t, ea4=s)

    NN = pygetm.mixing.get_buoyancy_frequency(s, t)
    print(NN.ma.min(), NN.ma.max())
    date += timedelta

    if itime % 10 == 0:
        # Update plot
        title.set_text(date.strftime('%Y-%m-%d %H:%M'))
        pc.set_array(sst.ma.ravel())
        pc_hf.set_array(shf.ma.ravel())

        fig.canvas.draw()

In [ ]:
fig, ax = matplotlib.pyplot.subplots()
NN.isel(z=-2).plot()